In [36]:
# 参与融合的模型结果，文件均为提交格式
# Bert-CRF
add_test0 = open('/home/liyunliang/CCL2022_CGED-8_Top1_project/model_fuse/Bert_CRF.txt').readlines()
# Bert-Bilstm-CRF
add_test1 = open('/home/liyunliang/CCL2022_CGED-8_Top1_project/model_fuse/Bert_Bilstm_CRF.txt').readlines()
# Pointer Generator Network
add_test2 = open('/home/liyunliang/CCL2022_CGED-8_Top1_project/model_fuse/Pointer_Generator_Net.txt').readlines()
# Gector
add_test3 = open('/home/liyunliang/CCL2022_CGED-8_Top1_project/model_fuse/Gector.txt').readlines()
# 错别字
add_test4 = open('/home/liyunliang/CCL2022_CGED-8_Top1_project/model_fuse/病句S.txt').readlines()
add_test5 = open('/home/liyunliang/CCL2022_CGED-8_Top1_project/model_fuse/易错字S.txt').readlines()
# 指针生成网络二阶段训练
add_test6 = open('/home/liyunliang/CCL2022_CGED-8_Top1_project/model_fuse/PGN_二阶段.txt').readlines()


In [38]:
# 获取统计融合结果的字典信息
fuse_test_dict = {}
""" fuse_test_dict为统计融合结果的字典，结构为：
    {
    id: {
        (错误类型, 错误位置): [包含该错误点的模型索引列表, 包含该错误点的模型输出的所有纠正结果的集合]
        }
    }
"""
for i, add_test in enumerate([add_test0, add_test1, add_test2, add_test3, add_test4, add_test5, add_test6]):
    # 遍历每个模型结果文件
    for add in add_test:
        # 遍历每行结果
        add = add.strip().split(',\t')
        assert len(add) in [2,4,5]
        if add[0] not in fuse_test_dict:
            fuse_test_dict[add[0]] = {}
        if len(add) == 2:
            # correct
            if 'correct' not in fuse_test_dict[add[0]]:
                fuse_test_dict[add[0]]['correct'] = [[], set()]
            # 列表挂上判定为correct的模型索引
            fuse_test_dict[add[0]]['correct'][0].append(i)
        # elif len(add) == 4:
        elif len(add) == 4 or i in [0, 1]:
            # R/W
            if (add[3], (add[1], add[2])) not in fuse_test_dict[add[0]]:
                fuse_test_dict[add[0]][(add[3], (add[1], add[2]))] = [[], set()]
            # 列表挂上包含该错误点的模型索引
            fuse_test_dict[add[0]][(add[3], (add[1], add[2]))][0].append(i)
        else:
            # S/M
            if (add[3], (add[1], add[2])) not in fuse_test_dict[add[0]]:
                fuse_test_dict[add[0]][(add[3], (add[1], add[2]))] = [[], set()]
            # 列表挂上包含该错误点的模型索引
            fuse_test_dict[add[0]][(add[3], (add[1], add[2]))][0].append(i)
            # 集合添加包含该错误点的模型输出的纠正结果
            fuse_test_dict[add[0]][(add[3], (add[1], add[2]))][1].add(add[4])
fuse_test_dict

{'1001': {('R', ('43', '43')): [[0, 1, 2, 3, 6], set()],
  ('M', ('51', '51')): [[3, 6], {'了'}],
  ('S', ('43', '43')): [[5], {'你'}]},
 '1002': {'correct': [[0, 1, 2, 3], set()],
  ('S', ('43', '43')): [[5], {'残'}],
  ('R', ('21', '27')): [[6], set()]},
 '1003': {'correct': [[0, 1, 6], set()],
  ('S', ('9', '9')): [[2], {'都'}],
  ('R', ('14', '14')): [[2], set()],
  ('R', ('7', '8')): [[3], set()]},
 '1004': {'correct': [[0, 1, 2, 6], set()],
  ('M', ('31', '31')): [[3], {'活动'}]},
 '1005': {('S', ('22', '22')): [[0, 1, 2, 3, 5, 6], {'全', '比', '超过'}],
  ('M', ('27', '27')): [[3], {'多'}]},
 '1006': {'correct': [[0, 1, 2, 3, 6], set()]},
 '1007': {('R', ('6', '6')): [[0, 1, 2, 3, 6], set()]},
 '1008': {'correct': [[0, 1, 2, 6], set()], ('R', ('12', '12')): [[3], set()]},
 '1009': {('S', ('2', '2')): [[0, 1, 3], {'次'}],
  ('M', ('9', '9')): [[0, 1], set()],
  ('R', ('17', '18')): [[0, 1], set()],
  ('S', ('1', '2')): [[2, 4], {'从那'}],
  ('R', ('2', '2')): [[6], set()]},
 '1010': {('M', ('2

In [39]:
# 获取融合结果为correct的样本id
correct_list = []
for id_, item in fuse_test_dict.items():
    # 不小于3个模型投票该样本为correct，则初步判定该样本为correct
    if 'correct' in item and len(item['correct'][0])>=3:
        correct_list.append(id_)
len(correct_list)

1623

In [40]:
"""
基于lang8+历年数据单步训练的指针生成网络模型，线上提交分数为0.3967；
由于指针网络适用于乱序类型的错误，并且在验证集的W类型指标的精确率较高，因此融合中直接将该模型输出的W类型错误判定为无误，并把与之重合的错误剔除掉；
删除start==end的W类型错误点；
如果某个判定为correct的样本中包含该模型输出的W类型错误，则将其从correct列表中剔除；
"""
# 剔除与Ptr生成的W类型错误点有overlap的错误点
for id_, item in fuse_test_dict.items():
    # 将每个样本的错误点按照位置的start进行排序
    new_item = sorted([(k, v) for k, v in item.items() if k != 'correct'], key=lambda x: int(x[0][1][0]))

    for i, char_item in enumerate(new_item):
        if char_item[0][0] == 'W' and char_item[0][1][0] == char_item[0][1][1]:
            # 删除start==end的W类型错误点
            print('delete W :', id_, char_item)
            fuse_test_dict[id_].pop(char_item[0])
        elif char_item[0][0] == 'W' and 2 in char_item[1][0]:
            # Pointer_Generator_Net识别出来的W类型错误
            print('Ptr W: ', id_, char_item)
            if id_ in correct_list:
                # 如果某个判定为correct的样本中包含该模型输出的W类型错误，则将其从correct列表中剔除
                print('remove id ', id_)
                correct_list.remove(id_)
            for j in range(i-1, -1, -1):
                # 从后往前找，把与之重合的错误剔除掉
                if int(new_item[j][0][1][1]) >= int(char_item[0][1][0]):
                    print('delete 重叠 :', id_, new_item[j])
                    fuse_test_dict[id_].pop(new_item[j][0])
            for j in range(i+1, len(new_item)):
                # 从前往后找，把与之重合的错误剔除掉
                if int(new_item[j][0][1][0]) <= int(char_item[0][1][1]):
                    print('delete 重叠 :', id_, new_item[j])
                    fuse_test_dict[id_].pop(new_item[j][0])
            print('')
# 查看剩下的判定为correct的样本个数
len(correct_list)

Ptr W:  1039 (('W', ('18', '25')), [[0, 2, 3, 6], set()])
delete 重叠 : 1039 (('R', ('18', '19')), [[1], set()])

Ptr W:  1122 (('W', ('2', '7')), [[1, 2], set()])
delete 重叠 : 1122 (('W', ('2', '10')), [[0], set()])
delete 重叠 : 1122 (('S', ('1', '2')), [[3], {'这是我'}])
delete 重叠 : 1122 (('R', ('2', '2')), [[6], set()])

Ptr W:  1246 (('W', ('14', '19')), [[0, 1, 2, 6], set()])
delete 重叠 : 1246 (('S', ('14', '19')), [[3], {'给别人添麻烦'}])

Ptr W:  1258 (('W', ('2', '10')), [[0, 2], set()])
delete 重叠 : 1258 (('R', ('2', '6')), [[3], set()])

Ptr W:  1444 (('W', ('50', '52')), [[1, 2, 6], set()])
delete 重叠 : 1444 (('S', ('50', '50')), [[0, 3], {'给'}])
delete 重叠 : 1444 (('M', ('51', '51')), [[0], set()])

Ptr W:  1647 (('W', ('6', '8')), [[0, 1, 2, 3, 6], set()])

Ptr W:  1703 (('W', ('7', '11')), [[2, 6], set()])
delete 重叠 : 1703 (('S', ('7', '8')), [[3], {'我对'}])
delete 重叠 : 1703 (('R', ('11', '11')), [[3], set()])

Ptr W:  1732 (('W', ('10', '11')), [[2, 6], set()])
delete 重叠 : 1732 (('S', ('1

1622

In [41]:
# 根据投票统计位置重合的错误点信息
from collections import defaultdict
error_pos_info = defaultdict(dict)
"""
error_pos_info:{
    id: {
        错误位置(start, end): {
            错误类型: 贡献该错误位置与错误类型的模型列表
        }
    }
}
"""

for id_, item in fuse_test_dict.items():
    for error, cnt in item.items():
        # 统计错误位置重合
        if error != 'correct':
            if str(error[1]) not in error_pos_info[id_]:
                error_pos_info[id_][str(error[1])] = defaultdict(list)
            error_pos_info[id_][str(error[1])][error[0]] += cnt[0]
error_pos_info

defaultdict(dict,
            {'1001': {"('43', '43')": defaultdict(list,
                          {'R': [0, 1, 2, 3, 6], 'S': [5]}),
              "('51', '51')": defaultdict(list, {'M': [3, 6]})},
             '1002': {"('43', '43')": defaultdict(list, {'S': [5]}),
              "('21', '27')": defaultdict(list, {'R': [6]})},
             '1003': {"('9', '9')": defaultdict(list, {'S': [2]}),
              "('14', '14')": defaultdict(list, {'R': [2]}),
              "('7', '8')": defaultdict(list, {'R': [3]})},
             '1004': {"('31', '31')": defaultdict(list, {'M': [3]})},
             '1005': {"('22', '22')": defaultdict(list,
                          {'S': [0, 1, 2, 3, 5, 6]}),
              "('27', '27')": defaultdict(list, {'M': [3]})},
             '1007': {"('6', '6')": defaultdict(list, {'R': [0, 1, 2, 3, 6]})},
             '1008': {"('12', '12')": defaultdict(list, {'R': [3]})},
             '1009': {"('2', '2')": defaultdict(list,
                          {'S': [0,

In [42]:
"""
基于统计位置重合的错误点信息，对位置重合的错误点进行筛选
筛选机制为：
1）将每个样本的错误点按照位置排序
2）顺序遍历每个错误点，将位置有重合的错误点放进一个列表中
3）对该重合列表中的错误点进行筛选，选择模型数目最高的错误位置，也即是某个错误位置有多少个模型判定，判定数目越高，越置信
4）在对错误位置进行筛选时，只考虑模型数目最高的2个位置差别>=1的情况
"""

for id_, item in error_pos_info.items():
    # 将每个id的错误位置按照start排序
    new_item = sorted([(k, v) for k, v in item.items()], key=lambda x: int(eval(x[0])[0]))
    for i, info in enumerate(new_item):
        if i != len(new_item)-1 and info[0] in error_pos_info[id_]:
            # 统计贡献当前错误位置的模型数目总和，包含不同的错误类型，并挂在tmp_list中
            tmp_list = [(info[0], sum([len(z) for z in info[1].values()]))]
            # 顺序遍历每个错误点，将位置有重合的错误点放进一个列表中
            for j in range(i+1, len(new_item)):
                if new_item[j][0] in error_pos_info[id_] and int(eval(new_item[j][0])[0]) <= int(eval(info[0])[1]):
                    tmp_list.append((new_item[j][0], sum([len(z) for z in new_item[j][1].values()])))
            # 对tmp_list进行排序，参与判定的模型数目越多，排序越靠前
            sorted_tmp_list = sorted(tmp_list, key=lambda x: x[1], reverse=True)
            # print(sorted_tmp_list)
            if len(sorted_tmp_list) > 1 and sorted_tmp_list[0][1] - sorted_tmp_list[1][1] >= 1:
                # 这种筛选只考虑模型数目差别>=1的情况
                for p in sorted_tmp_list[1:]:
                    print(error_pos_info[id_])
                    error_pos_info[id_].pop(p[0])
                    print(f'delete pos: id {id_}; pos {p[0]}')
                    print(error_pos_info[id_])
# error_pos_info

{"('2', '2')": defaultdict(<class 'list'>, {'S': [0, 1, 3], 'R': [6]}), "('9', '9')": defaultdict(<class 'list'>, {'M': [0, 1]}), "('17', '18')": defaultdict(<class 'list'>, {'R': [0, 1]}), "('1', '2')": defaultdict(<class 'list'>, {'S': [2, 4]})}
delete pos: id 1009; pos ('1', '2')
{"('2', '2')": defaultdict(<class 'list'>, {'S': [0, 1, 3], 'R': [6]}), "('9', '9')": defaultdict(<class 'list'>, {'M': [0, 1]}), "('17', '18')": defaultdict(<class 'list'>, {'R': [0, 1]})}
{"('40', '40')": defaultdict(<class 'list'>, {'R': [0, 1, 2, 3], 'S': [5]}), "('34', '34')": defaultdict(<class 'list'>, {'S': [2, 6]}), "('23', '24')": defaultdict(<class 'list'>, {'R': [3]}), "('40', '41')": defaultdict(<class 'list'>, {'R': [6]})}
delete pos: id 1015; pos ('40', '41')
{"('40', '40')": defaultdict(<class 'list'>, {'R': [0, 1, 2, 3], 'S': [5]}), "('34', '34')": defaultdict(<class 'list'>, {'S': [2, 6]}), "('23', '24')": defaultdict(<class 'list'>, {'R': [3]})}
{"('11', '11')": defaultdict(<class 'list'>

In [14]:
"""
接下来需要从语言模型困惑度PPL的角度对同一个位置上的多个错误类型进行筛选
首先需要初始化一个基于bert的语言模型MLM，用于计算困惑度
"""

from transformers import BertPreTrainedModel, BertModel, BertConfig, BertTokenizer
import sys
import os
sys.path.append('/home/liyunliang/CCL2022_CGED-8_Top1_project/model_fuse')
from bert_model import BertOnlyMLMHead
from torch.nn import CrossEntropyLoss
import torch.nn.functional as F
os.environ["CUDA_VISIBLE_DEVICES"] = '7'


class MLM(BertPreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        self.bert = BertModel(config)
        self.cls = BertOnlyMLMHead(config)
        self.init_weights()
        self.loss = CrossEntropyLoss()

    def forward(
            self,
            input_ids=None,
            attention_mask=None,
            token_type_ids=None,
            target=None,

    ):
        outputs = self.bert(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
        )
        sequence_output = outputs[0]
        batch_size, seq_len, hidden_dim = sequence_output.size()
        # 将mask处的预测概率取出
        x_mask = (input_ids == 103).unsqueeze(-1)
        masked_output = torch.masked_select(sequence_output, x_mask).view(
            batch_size, -1, hidden_dim)
        prediction_scores = self.cls(masked_output)
        prediction_scores = F.softmax(prediction_scores, dim=-1).squeeze(1)
        return prediction_scores
config = BertConfig.from_pretrained('/home/liyunliang/pretrained_model/chinese_wwm_ext_pytorch/')
ppl_model = MLM.from_pretrained('/home/liyunliang/pretrained_model/chinese_wwm_ext_pytorch/', config=config).to('cuda')
tokenizer = BertTokenizer.from_pretrained('/home/liyunliang/pretrained_model/chinese_wwm_ext_pytorch/')

/home/liyunliang/anaconda3/envs/cged/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2022-09-21 16:13:38.421402: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
Some weights of the model checkpoint at /home/liyunliang/pretrained_model/chinese_wwm_ext_pytorch/ were not used when initializing MLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing MLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT exp

In [15]:
"""
定义计算局部困惑度PPL的函数
融合结果中存在同一个位置有多个可能的错误类型，那么可以采用局部困惑度来进行筛选
比如某个位置有S和R的错误类型，可以通过计算这两种错误修改前后的困惑度降低值delta来衡量哪种错误类型更有可能是对的
如果一种错误类型修改前后的困惑度降低值delt越大(>0)，说明修改前困惑度高，修改后困惑度降低了很多，句子变得更准确，那么该错误类型的结果趋向于是正确的结果
"""

from copy import deepcopy
import torch
from torch.nn.utils.rnn import pad_sequence
import numpy as np

def cal_ppl(left: str, wrong_span: str, right: str):
    """
    @left: wrong_span左边原封不动的部分
    @wrong_span: 需要使用[MASK]计算困惑度的局部子串
    @right: wrong_span右边原封不动的部分
    """
    examples = []
    left, wrong_span, right = list(left), list(wrong_span), list(right)
    for i, token in enumerate(wrong_span):
        new_wrong_span = deepcopy(wrong_span)
        new_wrong_span[i] = "[MASK]"
        sentence = left + new_wrong_span + right
        pos = len(left) + i
        target = torch.tensor([tokenizer.convert_tokens_to_ids(token)]).to('cuda')
        tokenize_output = tokenizer(''.join(sentence))
        info = {}
        info["sentence"] = sentence
        info['input_ids'] = tokenize_output['input_ids']
        info['token_type_ids'] = tokenize_output['token_type_ids']
        info['attention_mask'] = tokenize_output['attention_mask']
        info["target"] = target
        examples.append(info)

    input_ids = pad_sequence([torch.tensor(feature['input_ids'], dtype=torch.long) for feature in examples], batch_first=True, padding_value=0)
    attention_mask = pad_sequence([torch.tensor(feature['attention_mask'], dtype=torch.long) for feature in examples], batch_first=True, padding_value=0)
    token_type_ids = pad_sequence([torch.tensor(feature['token_type_ids'], dtype=torch.long) for feature in examples], batch_first=True, padding_value=0)
    labels = torch.stack([feature['target'] for feature in examples])
    
    with torch.no_grad():
        prediction_scores = ppl_model(
            input_ids=input_ids.to('cuda'), attention_mask=attention_mask.to('cuda'),
            token_type_ids=token_type_ids.to('cuda'))
        res = prediction_scores.gather(1, labels)
    # 计算困惑度的公式
    return -1 * np.sum([np.log(float(item)) for item in res]) / len(res)

# cal_ppl('我爱', '我的', '祖国')
cal_ppl('我爱', '你的', '祖国')

1.1397987631721487

In [16]:
# 获取测试集id和text的映射字典

import tqdm
test_data = open('/home/liyunliang/CGED_Task/dataset/cged2022/cged2022-test.jsonl').readlines()
ppl_list = {}
import json
for line in tqdm.tqdm(test_data, total=len(test_data)):
    line = json.loads(line)
    ppl_list[line['id']] = line['text']

100%|██████████| 3767/3767 [00:00<00:00, 172718.50it/s]


In [17]:
"""
判断一个位置的改动带来的ppl降低值，如果ppl降低，则该改动趋向于正确：>0说明困惑度降低了，<0说明困惑度增大了
由于W类型错误较复杂，因此不考虑W类型修改前后的困惑度降低值
"""

def delta_ppl(id_, position, error_type):
    """
    @id_: 样本id
    @position: 位置信息
    @error_type: 错误类型
    """
    item = fuse_test_dict[id_][(error_type, position)]
    text = ppl_list[id_]
    left, right = int(position[0])-1, int(position[1])
    # 计算修改前错误位置的困惑度
    ppl0 = cal_ppl(text[:left], text[left:right], text[right:])
    # print(ppl0)
    if error_type == 'S':
        # S类型的错误，计算替换后的困惑度，然后统计困惑度降低值
        chars = list(item[1])
        if chars:
            ppls = []
            for char in chars:
                # 遍历每个纠正备选项，计算替换后的困惑度
                tmp_ppl = cal_ppl(text[:left], char, text[right:])
                if ppl0-tmp_ppl <= 0:
                    # 如果困惑度降低值<=0，说明该纠正结果使得困惑度变大了，纠正结果趋向于是错误的，因此删除该纠正结果
                    print('remove ', char)
                    print(fuse_test_dict[id_][(error_type, position)])
                    fuse_test_dict[id_][(error_type, position)][1].remove(char)
                    print(fuse_test_dict[id_][(error_type, position)])
                ppls.append(tmp_ppl)
            # print(ppls)
            if ppls:
                # 选取修改后困惑度最小的那个
                min_ppl = min(ppls)
            else:
                return None
        else:
            return None

    elif error_type == 'M':
        # M类型的错误，计算补充缺失信息后的困惑度，然后统计困惑度降低值
        chars = list(item[1])
        if chars:
            ppls = []
            for char in chars:
                # 遍历每个纠正备选项，计算补充缺失信息后的困惑度
                tmp_ppl = cal_ppl(text[:left], char, text[right:])
                if ppl0-tmp_ppl <= 0:
                    # 如果困惑度降低值<=0，说明该纠正结果使得困惑度变大了，纠正结果趋向于是错误的，因此删除该纠正结果
                    print('remove ', char)
                    print(fuse_test_dict[id_][(error_type, position)])
                    fuse_test_dict[id_][(error_type, position)][1].remove(char)
                    print(fuse_test_dict[id_][(error_type, position)])
                ppls.append(tmp_ppl)
            # print(ppls)
            if ppls:
                # 选取修改后困惑度最小的那个
                min_ppl = min(ppls)
            else:
                return None
        else:
            return None
    elif error_type == 'R':
        # R类型的错误，计算去除冗余信息后的困惑度，然后统计困惑度降低值，这里采用冗余字段前后各一个字符作为计算局部困惑度的span
        min_ppl = cal_ppl(text[:left-1], text[left-1]+text[right], text[right+1:])
    # 返回修改前后困惑度降低值
    return ppl0-min_ppl

# delta_ppl('1035', ('48', '48'), 'M')

In [43]:
"""
当同一个位置有多个错误类型，首先通过贡献该错误类型的模型数目来筛选，选择更多模型判定该错误类型的选项
当同一个位置的多个错误类型中，参与判定的模型数量差别不大，则需要通过计算同一个位置的不同错误点修改前后的困惑度降低值，筛选出困惑度降低值更大的错误点
"""

import copy
from collections import defaultdict
error_pos_info = defaultdict(dict)

for id_, item in fuse_test_dict.items():
    for error, cnt in item.items():
        # 统计错误位置重合
        if error != 'correct':
            if str(error[1]) not in error_pos_info[id_]:
                error_pos_info[id_][str(error[1])] = defaultdict(list)
            error_pos_info[id_][str(error[1])][error[0]] += cnt[0]
    for pos, item in error_pos_info[id_].items():
        if len(item) > 1:
            # 同一个位置上有多个错误类型
            print(len(item), '\t', id_, pos, item)
            if len(item) == 2:
                # 同一个位置上有2个错误类型，对错误点按照参与判定的模型数目倒排
                sorted_item = sorted(item.items(), key=lambda x: len(x[1]), reverse=True)
                if len(sorted_item[0][1]) - len(sorted_item[1][1]) >= 2:
                    # 当前者的模型数目比后者的模型数目超出2的时候（包含2），则可以认定前者是正确的错误类型，删除后者
                    error_pos_info[id_][pos].pop(sorted_item[1][0])
                    print('delete: ', len(item), '\t', id_, pos, item)
                else:
                    # 当前者的模型数目比后者的模型数目未超出2的时候
                    if sorted_item[0][0] in ['M', 'S', 'R'] and sorted_item[1][0] in ['M', 'S', 'R']:
                        # 2者均不是W类型，删除困惑度降低值更小的那个
                        a = delta_ppl(id_, eval(pos), sorted_item[0][0])
                        b = delta_ppl(id_, eval(pos), sorted_item[1][0])
                        if a and b:
                            if a > b:
                                error_pos_info[id_][pos].pop(sorted_item[1][0])
                                print('delete: ', len(item), '\t', id_, pos, item)
                            elif a < b:
                                error_pos_info[id_][pos].pop(sorted_item[0][0])
                                print('delete: ', len(item), '\t', id_, pos, item)
                    elif sorted_item[0][0] in ['M', 'S', 'R']:
                        # 前者不是W类型，后者是W类型
                        a = delta_ppl(id_, eval(pos), sorted_item[0][0])
                        if a and a > 0:
                            error_pos_info[id_][pos].pop(sorted_item[1][0])
                            print('delete: ', len(item), '\t', id_, pos, item)
                    elif sorted_item[1][0] in ['M', 'S', 'R']:
                        # 前者是W类型，后者不是W类型
                        b = delta_ppl(id_, eval(pos), sorted_item[1][0])
                        if b and b>0:
                            error_pos_info[id_][pos].pop(sorted_item[0][0])
                            print('delete: ', len(item), '\t', id_, pos, item)
            elif len(item) == 3:
                # 同一个位置上有3个错误类型，只考虑模型数目层面的筛选
                sorted_item = sorted(item.items(), key=lambda x: x[1], reverse=True)
                if len(sorted_item[0][1]) - len(sorted_item[1][1]) >= 2:
                    error_pos_info[id_][pos].pop(sorted_item[1][0])
                    error_pos_info[id_][pos].pop(sorted_item[2][0])
                    print('delete: ', len(item), '\t', id_, pos, item)
                else:
                    pass
            else:
                print('error')
# error_pos_info

2 	 1001 ('43', '43') defaultdict(<class 'list'>, {'R': [0, 1, 2, 3, 6], 'S': [5]})
delete:  1 	 1001 ('43', '43') defaultdict(<class 'list'>, {'R': [0, 1, 2, 3, 6]})
2 	 1009 ('2', '2') defaultdict(<class 'list'>, {'S': [0, 1, 3], 'R': [6]})
delete:  1 	 1009 ('2', '2') defaultdict(<class 'list'>, {'S': [0, 1, 3]})
2 	 1015 ('40', '40') defaultdict(<class 'list'>, {'R': [0, 1, 2, 3], 'S': [5]})
delete:  1 	 1015 ('40', '40') defaultdict(<class 'list'>, {'R': [0, 1, 2, 3]})
2 	 1021 ('2', '2') defaultdict(<class 'list'>, {'R': [0, 1], 'S': [2, 6]})
delete:  1 	 1021 ('2', '2') defaultdict(<class 'list'>, {'S': [2, 6]})
2 	 1047 ('29', '29') defaultdict(<class 'list'>, {'R': [0, 1], 'S': [2]})
delete:  1 	 1047 ('29', '29') defaultdict(<class 'list'>, {'S': [2]})
2 	 1051 ('5', '5') defaultdict(<class 'list'>, {'R': [0, 1, 3], 'S': [6]})
delete:  1 	 1051 ('5', '5') defaultdict(<class 'list'>, {'R': [0, 1, 3]})
2 	 1054 ('46', '46') defaultdict(<class 'list'>, {'R': [1, 6], 'S': [5]})
d

In [44]:
# 生成提交的融合结果文件
import pandas as pd
output = {'id': [], 'position_pred': []}
count = 0
for id_, item in fuse_test_dict.items():
    output['id'].append(id_)
    position_pred = []

    for error, cnt in item.items():
        if len(cnt[0]) >= 3 and error == 'correct' and id_ in correct_list:
            # 不小于3个模型投票该样本为correct，且id在correct_list中，则判定该样本为correct
            count += 1
            position_pred = []
            break
        elif len(cnt[0]) >= 1 and error != 'correct' and error[0] in error_pos_info[id_][str(error[1])]:
            if error[0] in ['S', 'M'] and list(cnt[1]) != []:
                position_pred.append((error[0], error[1][0], error[1][1], list(cnt[1])))
            else:
                position_pred.append((error[0], error[1][0], error[1][1]))
    output['position_pred'].append(position_pred[:])

print(count)
# pd.DataFrame(output)
res = []
for id_, ans in zip(output['id'], output['position_pred']):
    if ans == []:
        res.append(',\t'.join([id_, 'correct']))
    else:
        for item in ans:
            if len(item) == 3:
                # R/W或者未输出纠正结果的S/M
                res.append(',\t'.join([id_, str(item[1]), str(item[2]), str(item[0])]))
            else:
                new_ans = []
                for char in item[3]:
                    if len(char) >= 3 or len(set(char))/len(char)<=0.5:
                        # 去除纠正结果中长度>=3的（根据验证集的badcase，纠正结果长度过长趋向于是错误的），以及字符重复度过高的
                        continue
                    new_ans.append(char)
                if len(new_ans) > 1:
                    # print(id_, item)
                    res.append(',\t'.join([id_, str(item[1]), str(item[2]), str(item[0]), ', '.join(new_ans)]))
                elif len(new_ans) == 1:
                    res.append(',\t'.join([id_, str(item[1]), str(item[2]), str(item[0]), new_ans[0]]))
                else:
                    res.append(',\t'.join([id_, str(item[1]), str(item[2]), str(item[0])]))
open('/home/liyunliang/CCL2022_CGED-8_Top1_project/cged_pred_fuse_09-21.txt', 'w').write('\n'.join(res))
print(len(set([item.split(',\t')[0] for item in res])))

1622
3767
